#### Script for reading and storing data

idea being - instead of loading all text passages in memory  with f.open(), stream data and only store limited passages in mem


use generators: generator.next() gives me the passage, for loop handles this automatically

In [30]:
!curl -L -o dataset.zip https://www.kaggle.com/api/v1/datasets/download/shubhammaindola/harry-potter-books
!unzip dataset.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2334k  100 2334k    0     0  7634k      0 --:--:-- --:--:-- --:--:-- 7634k
Archive:  dataset.zip
replace 01 Harry Potter and the Sorcerers Stone.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [31]:
paths = !ls | grep ".txt"
book_1 = paths[0]
print(paths)

['01 Harry Potter and the Sorcerers Stone.txt', '02 Harry Potter and the Chamber of Secrets.txt', '03 Harry Potter and the Prisoner of Azkaban.txt', '04 Harry Potter and the Goblet of Fire.txt', '05 Harry Potter and the Order of the Phoenix.txt', '06 Harry Potter and the Half-Blood Prince.txt', '07 Harry Potter and the Deathly Hallows.txt']


#### Spacy

In [32]:
import spacy
import pandas as pd
nlp = spacy.load("en_core_web_sm")

In [38]:
def stream_passages(file_path,context_size=1024):
    with open(file_path, "r", encoding="utf-8") as f:
        buffer = ""
        for line in f:
            buffer += line
        while len(buffer) > context_size:
            yield buffer[:context_size]
            buffer = buffer[context_size:]
        if buffer:
            yield buffer

def analyze_text(text:str,nlp,df:pd.DataFrame=None) -> pd.DataFrame:
    doc = nlp(text)
    rows = []

    for token in doc:
        rows.append({
            "token":token.text,
            "POS":token.pos_,
            "ENT":token.ent_type_
        })

    df_rows = pd.DataFrame(rows)
    if df is None:
        return df_rows
    else:
        df = pd.concat([df, df_rows], ignore_index=True)
        return df
    

def process_corpora():
    batch_data = stream_passages(book_1)
    df = pd.DataFrame()
    for batches in batch_data:
        df = analyze_text(batches,nlp,df)

    df.to_csv("output.csv", index=False)    

In [ ]:
process_corpora()
!cat output.csv